# Здравствуйте

Продолжаю улучшать свои навыки и знания.
Предлагаю ознакомиться с мини проектом "Анализом лояльности пользователей", в котором привожу решение простых типовых задач.

Данные для мини проектов можно найти на сайте https://www.kaggle.com

С уважением Осокин Максим


# Задача

* Необходимо получить данные для тестирования программы лояности.
* На платформе размещены товары нескольких крупных брендов, неоходимо выбрать самых лояных покупателей, нужны пользователи которые в долгосрочной перспективе компенсируют скидку своей лояльностью.

Данные которые понадобятся: 

* tc - id_пользователя 
* art_sp - название бренда

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://stepik.org/media/attachments/course/111949/lesson_3_data_1_.csv', encoding='windows-1251')

In [4]:
df.head()

,Unnamed: 0,tk,pl,dia,hs,tc,cta,id_art,id_subsubfam,id_subfam,...,id_famn,id_seccion,id_subagr,id_agr,vta,uni,id_artn,art_sp,fam_sp,fam_en
0,1242,120071109002055793,1,2007/11/09,0.505729,110000761,11000076,21895,101070640100,1010706401,...,10107064,10107,101,1,0.68,1.0,21895,MARAVILLA 500 G Store_Brand,PASTA ALIMENTICIA SE,pasta
1,1243,120070601004053633,1,2007/06/01,0.588519,110000761,11000076,21816,101070640100,1010706401,...,10107064,10107,101,1,0.38,1.0,21816,FIDEO CABELLIN 500 G Store_Brand,PASTA ALIMENTICIA SE,pasta
2,1244,120070928004076410,1,2007/09/28,0.785498,28491841,2849184,562840,101070640100,1010706401,...,10107064,10107,101,1,1.55,2.0,562840,SPAGUETTI Nє 5 500 G Brand_1,PASTA ALIMENTICIA SE,pasta
3,1245,120070616003074261,1,2007/06/16,0.550961,95931501,9593150,28914,101070640100,1010706401,...,10107064,10107,101,1,1.03,2.0,28914,FIDEO FIDEUБ 500 Brand_7,PASTA ALIMENTICIA SE,pasta
4,1246,120070519001018477,1,2007/05/19,0.804954,93265591,9326559,159867,101070640100,1010706401,...,10107064,10107,101,1,1.09,1.0,159867,MACARRONES GRATINAR 5 Brand_2,PASTA ALIMENTICIA SE,pasta


Для удобства создам переменную user_df , поместив в нее строки tc и art_sp

In [5]:
user_df = df[['tc', 'art_sp']]

In [6]:
user_df

,tc,art_sp
0,110000761,MARAVILLA 500 G Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand
2,28491841,SPAGUETTI Nє 5 500 G Brand_1
3,95931501,FIDEO FIDEUБ 500 Brand_7
4,93265591,MACARRONES GRATINAR 5 Brand_2
...,...,...
48124,45518841,FIDEOS 0 500 G Brand_4
48125,110824211,PLUMAS 3 500 G Brand_4
48126,1408670389,MACARRONES 500 G Store_Brand
48127,1408670389,SPAGHETTI 500 G Store_Brand


In [7]:
user_df = user_df.rename(columns={'tc': 'user_id', 'art_sp': 'brand_info'})
# переименовал в более удобные для работы названия колонок

In [8]:
user_df.head()

,user_id,brand_info
0,110000761,MARAVILLA 500 G Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand
2,28491841,SPAGUETTI Nє 5 500 G Brand_1
3,95931501,FIDEO FIDEUБ 500 Brand_7
4,93265591,MACARRONES GRATINAR 5 Brand_2


* Разделение строк - split - метод

для дальнейшей работы с данными (разделю колонку brand_info на колонки brand_name и brand_info)

In [9]:
def split_brand(brand_name_data):
    return brand_name_data.split(' ')[-1]

In [10]:
brnad_name = 'MARAVILLA 500 G Store_Brand'

In [11]:
brnad_name.split(' ')[-1]

'Store_Brand'

In [12]:
user_df['brand_name'] = user_df.brand_info.apply(split_brand)

In [13]:
# user_df['brand_name'] = user_df.brand_info.apply(lambda x: x.split(' ')[-1])
# можно использовать lambda функцию, чтобы разрезать строку по пробелу и взять последний элемент

In [14]:
user_df.head()

,user_id,brand_info,brand_name
0,110000761,MARAVILLA 500 G Store_Brand,Store_Brand
1,110000761,FIDEO CABELLIN 500 G Store_Brand,Store_Brand
2,28491841,SPAGUETTI Nє 5 500 G Brand_1,Brand_1
3,95931501,FIDEO FIDEUБ 500 Brand_7,Brand_7
4,93265591,MACARRONES GRATINAR 5 Brand_2,Brand_2


**Проверю гипотезу о наличии в наших данных лояльных пользователей.**


Мне интересно узнать id_пользователей которые совершают покупки одного бренда. 
* сгрупирую покупателей по количеству покупок
* задам релевантное имя колонке "покупки" = "purchases"
* выберу пользователе которые совершили 5 или более покупок


In [15]:
users_purchases = user_df.groupby('user_id', as_index=False)\
    .agg({'brand_name': 'count'})\
    .rename(columns={'brand_name': 'purchases'})\
    .query('purchases >= 5')

In [16]:
users_purchases.purchases.describe()
#.query('purchases') -  50% пользователей совершили 5 покупокупок
#.query('purchases >= 5') довил условие выбоа пользователей совершивших 5 и более покупок
#среднеквадратическое количество покупок одного бренда = 5,32
#среднее арифметическое количество покупок = 9,32
#на мой взгляд, все логично и критерий для выборки можно оставить прежним 

count    3383.000000
mean        9.320130
std         5.623993
min         5.000000
25%         6.000000
50%         7.000000
75%        11.000000
max        60.000000
Name: purchases, dtype: float64

**Разведочный анализ данных выполнил. Перехожу к решению задачи.**
* узнаю количество покупок одного бренда для каждого из пользователя.сохраню в переменную "brand_purchases"
* Узнаю для каждого пользователя любимый бренд. сохраню в переменную "purchases_lovely_brand_df"

In [17]:
brand_purchases = user_df.groupby(['user_id', 'brand_name'], as_index=False) \
    .agg({'brand_info' : 'count'}) \
   
# группирую данные user_id и brand_name и считаю количество покупок каждого бренда
# получил количество покупокупок совершенных пользователей в кадом из брендов более 5 позиций
brand_purchases

,user_id,brand_name,brand_info
0,-1236394515,Brand_4,1
1,1031,Brand_3,1
2,1031,Store_Brand,5
3,4241,Brand_4,3
4,4241,Store_Brand,2
...,...,...,...
18183,1408817589,Store_Brand,2
18184,1408825059,Brand_1,1
18185,1408832719,Brand_4,3
18186,1408840919,Store_Brand,1


In [18]:
purchases_lovely_brand_df = user_df.groupby(['user_id', 'brand_name'], as_index=False) \
    .agg({'brand_info' : 'count'}) \
    .sort_values(['user_id', 'brand_info'], ascending=[False, False])\
    .groupby('user_id') \
    .head(1)\
    .rename(columns={'brand_name' :'lovely_brand', 'brand_info' : 'lovely_brand_purchases' })

* группировал(сортировал)) пользователей и количество покупок по убыванию
* далее для каждого пользователя сгруппирую максимальное количество покупок товаров одного бренда (ascending=[False, False])
* метод head после группировки выведет указанную строку
* присвоил колонкам релевантные названия

Выведу для каждого пользователя число уникальных брендов, результат сохраню в переменую users_unique_brands

In [19]:
users_unique_brands = user_df.groupby('user_id', as_index=False)\
    .agg({'brand_name': pd.Series.nunique})\
    .rename(columns={'brand_name' : 'unique_brands'})


#pd.Series.nunique -расчет количества уникальных брендов

* merge метод для сведения датафреймов
* ??pd.DataFrame.merge - вывод документации

In [20]:
users_purchases
# узнал сколько всего покупок совершают пользовали и выбрал пользователей совершивших 5 и более покупок

,user_id,purchases
1,1031,6
2,4241,5
11,25971,7
14,40911,27
16,45181,5
...,...,...
11740,1408767189,5
11750,1408783189,10
11751,1408783379,6
11754,1408798879,8


In [21]:
users_unique_brands
# посчитал число уникальных брендов для каждого пользователя

,user_id,unique_brands
0,-1236394515,1
1,1031,2
2,4241,2
3,17311,1
4,17312,2
...,...,...
11759,1408817589,2
11760,1408825059,1
11761,1408832719,1
11762,1408840919,1


In [22]:
purchases_lovely_brand_df
# посчитал для каждого пользователя число покупок товаров любимого бренда

,user_id,lovely_brand,lovely_brand_purchases
18187,1408849249,Store_Brand,1
18186,1408840919,Store_Brand,1
18185,1408832719,Brand_4,3
18184,1408825059,Brand_1,1
18183,1408817589,Store_Brand,2
...,...,...,...
6,17312,Brand_1,1
5,17311,Brand_4,2
3,4241,Brand_4,3
2,1031,Store_Brand,5


In [84]:
loyalty_users=users_purchases\
    .merge(users_unique_brands, on='user_id')\
    .merge(purchases_lovely_brand_df, on='user_id')

# user_id - id_покупателя
# purchases - число покупупок 
# unique_brands - число уникальных брендов в которых совершались покупки
# lovely_brand - любимый бренд
# lovely_brand_purchases - число покупок на любимый бренд
loyalty_users

,user_id,purchases,unique_brands,lovely_brand,lovely_brand_purchases
0,1031,6,2,Store_Brand,5
1,4241,5,2,Brand_4,3
2,25971,7,2,Store_Brand,5
3,40911,27,5,Brand_4,19
4,45181,5,4,Store_Brand,2
...,...,...,...,...,...
3378,1408767189,5,1,Brand_4,5
3379,1408783189,10,2,Store_Brand,8
3380,1408783379,6,2,Brand_1,4
3381,1408798879,8,3,Store_Brand,4


Переименую имена колонок на русский, сохраню полученные результаты на отдельный лист в эксель loyalty_users_top.xlsx

In [86]:
loyalty_users_top =loyalty_users.rename(columns={'user_id': 'id_покупателя', \
                                                 'purchases': 'число покупупок', \
                                                 'unique_brands' : 'число уникальных брендов в которых совершались покупки',\
                                                 'lovely_brand' : 'любимый бренд',\
                                                 'lovely_brand_purchases' : 'число покупок на любимый бренд'})

with pd.ExcelWriter("loyalty_users_top.xlsx") as writer:
    loyalty_users_top.to_excel(writer) 

In [29]:
loyalty_users.lovely_brand_purchases.describe()

count    3383.000000
mean        7.069761
std         4.692481
min         1.000000
25%         4.000000
50%         6.000000
75%         8.000000
max        55.000000
Name: lovely_brand_purchases, dtype: float64